In [1]:
import json
import os
import pandas as pd
from IPython.display import JSON
import sqlite3
from termcolor import colored

# Workshop 5: Router Agent with Multiple Function Tools

In this final workshop, we will integrate the various function calls we have developed in previous workshops into a single, cohesive router agent using Azure OpenAI. This router agent will be capable of handling complex user requests by utilizing multiple predefined function tools.

We will cover the following steps:

1. **Set Up the Environment:**
   - Import the necessary libraries and helper modules containing our predefined function tools.
   - Configure the Azure OpenAI client using the API key and endpoint.
   - Connect to the SQLite database containing employee information to ensure it is accessible for querying employee details.

2. **Define System and Tools:**
   - Create a system message to set the context for the assistant, ensuring it understands its role and capabilities.
   - Utilize predefined JSON objects for function calls, including detailed descriptions to ensure precise interactions and data retrieval from the database.
   - List and map the predefined functions (e.g., scheduling meetings, sending emails, querying the database) and create tool definitions for these functions.

3. **User Interaction and Integration:**
   - Provide an example of a user interacting with the router agent, demonstrating how the agent can handle multiple tasks using the predefined function tools based on the user's requests.
   - Implement an interactive chat loop where the agent processes user inputs, executes the necessary actions using the defined functions, and responds accordingly.

Through this workshop, you will see how to integrate and utilize multiple function tools within a single agent to handle complex and multi-step user requests efficiently. This demonstration will show the practical application of combining various predefined function calls to create a robust and versatile assistant.


In [2]:
from openai import AzureOpenAI
from helpers.tools import func_tools, schedule_meeting, send_email, ask_database
from helpers.utilities import *

Tool: creating an email: Email to: recipient@example.com, Subject: Meeting Reminder, Body: This is a reminder for our meeting tomorrow., CC: cc1@example.com, cc2@example.com...



In [13]:
client = AzureOpenAI(
    azure_endpoint="https://<ENDPOINT>.openai.azure.com/",
    api_key="<API_KEY>",
    api_version="2024-02-15-preview"
)

In [4]:
conn = sqlite3.connect("employees.db")
cursor = conn.cursor()
print("Opened database successfully")

Opened database successfully


In [5]:
available_functions = {
    "schedule_meeting": schedule_meeting,
    "send_email": send_email,
    "ask_database": ask_database,
}

In [6]:
tools = list(func_tools.values())
JSON(func_tools)

<IPython.core.display.JSON object>

In [11]:
system_prompt = """**System Prompt: Microsoft Assistant Chatbot**

**Role:** You are a chatbot designed to assist users within the Microsoft ecosystem. You can converse with users, answer their questions using your internal knowledge, and perform specific tasks using your function tools.

**Capabilities:**
1. **Send an Email:** Compose and send emails on behalf of the user.
2. **Schedule a Meeting:** Arrange meetings and add them to calendars.
3. **Search Employee Information:** Query the Employee table in SQL to retrieve information about employees.

**Chain of Thoughts:**

1. **Understand the User Request:**
   - Carefully read and comprehend the user's request.
   - Identify the main tasks and any subtasks.

2. **Task Identification:**
   - Determine the primary tasks (e.g., sending an email, scheduling a meeting).
   - Recognize any dependencies (e.g., needing an email address before sending an email).

3. **Gather Necessary Information:**
   - Check if all required information is provided.
   - If any information is missing, identify the steps to acquire it (e.g., querying the Employee table).

4. **Execute Tasks Sequentially:**
   - **Step 1:** For sending an email:
     - Check if the recipient's email address is provided.
     - If not, query the Employee table for the email address.
     - Construct the email with the given message and send it.
     - Confirm the email was sent successfully.
   - **Step 2:** For scheduling a meeting:
     - Use the provided details (date, time, participants) to schedule the meeting.
     - Confirm the meeting was scheduled successfully.

**Example Workflow:**

- **User Request:** "Please send an email to Ben Almo saying that I will be available for our meeting and schedule a meeting with him on September 8, 2025, at 08:00 AM."

  **Chain of Thoughts:**
  - **Step 1:** Identify tasks: Send an email and schedule a meeting.
  - **Step 2:** Check if Ben Almo's email address is provided.
    - If not, query the Employee table for Ben Almo's email address.
  - **Step 3:** Construct the email with the message: "I will be available for our meeting."
  - **Step 4:** Send the email and confirm success.
  - **Step 5:** Schedule a meeting on September 8, 2025, at 08:00 AM with Ben Almo.
  - **Step 6:** Confirm the meeting was scheduled successfully.

**General Guidelines:**
- **Conversational Engagement:** Maintain a friendly and professional tone.
- **Precision:** Ask for specific missing information if required.
- **Confirmation:** Provide confirmation messages after successfully completing each task.
- **Error Handling:** Notify the user if any issues arise and provide possible solutions.

**IMPORTANT:** Each time you use a tool, explicitly state your actions (e.g., "using tool", "I need to use __ for ___"). if you will not create a Thought of what is the next step I WILL BE MAD!
"""

- can you tell me the names of managers in our company?
- can you give me the email address of Bob Brown?
- ok send to Bob Brown an email tell him that I will be late today.
- can you also schedule a meeting with him?
-  yes, so schedule it to 08:00 for the 21 to september

more complex example:
- """send an email to all managers say "Dear Managers, as we conclude this year, I wish next year to be as successful as this one." and schedule a meeting with all of them on 03/06/2024 at 08:00am """

In [12]:

memory = []
memory = append_to_history("system", system_prompt, memory)
print("---------------------------------------------------\nstarting conversation:\n------------------------------------------------------------")
while True:
    user_input = input("user: ")
    if user_input == "END":
        break;
    memory = append_to_history("user", user_input, memory)
    response = create_chat_response(client, tools, memory)
    memory.append(response.choices[0].message)
    using_tool = response.choices[0].message.tool_calls
    chain = False
    if using_tool:
        pretty_print(response)
        while using_tool:
            chain = True
            tool_calls = response.choices[0].message.tool_calls[0]
            function_name = tool_calls.function.name
            function_arguments = json.loads(tool_calls.function.arguments)
            function_to_call = available_functions[function_name]
            if function_name == "ask_database":
                function_arguments['conn'] = conn
                function_response = function_to_call(**function_arguments)
                memory = append_to_history(role="tool", content=function_response, tool_id=tool_calls.id,
                                           function_name=function_name, memory=memory)
            else:
                function_response = function_to_call(**function_arguments)
                memory = append_to_history(role="tool", content=function_response, tool_id=tool_calls.id,
                                           function_name=function_name, memory=memory)

            response = create_chat_response(client=client, tools=tools, memory=memory)
            pretty_print(response)
            memory.append(response.choices[0].message)
            using_tool = response.choices[0].message.tool_calls
    else:
        pretty_print(response)

---------------------------------------------------
starting conversation:
------------------------------------------------------------


user:  send an email to all managers say "Dear Managers, as we conclude this year, I wish next year to be as successful as this one." and schedule a meeting with all of them on 03/06/2024 at 08:00am


assistant: To complete your request, I need to perform two tasks: sending an email and scheduling a meeting. 

First, I will use the `ask_database` function to retrieve the email addresses of all managers from the Employees database. 

Let's start with that.

Tool: asking database for: SELECT email_address FROM Employees WHERE role = 'Manager'

assistant: I have successfully retrieved the email addresses of all managers. Now, I will use the `send_email` function to send an email to all of them with the message "Dear Managers, as we conclude this year, I wish next year to be as successful as this one." Let's proceed with that.
Tool: creating an email: Email to: ['john.doe@example.com', 'bob.brown@example.com', 'eve.miller@example.com', 'hank.anderson@example.com', 'kathy.white@example.com', 'nina.thompson@example.com', 'quincy.robinson@example.com', 'tina.walker@example.com', 'wendy.young@example.com', 'zane.green@example.com'], Subject: End of Year Message, Body: Dear Managers, as we c

user:  END


# Challenge: Implement Additional Tools for the Router Agent

In this challenge, you will expand the capabilities of our router agent by adding a new function tool. This exercise will help you understand how to integrate additional functionality into the agent, enhancing its ability to handle a wider range of user requests.

## Challenge Objective

1. **Understand the Existing Framework:**
   - Review the predefined function tools and their integration into the router agent.
   - Understand the structure of the function call JSON objects and how they interact with the agent.

2. **Implement a New Function Tool:**
   - Create a new function tool to manage a to-do list by adding tasks and retrieving the list of tasks.
   - Ensure the tool is well-documented with a clear description and parameter definitions.

### Tool: To-Do List Manager

Create a tool that allows the user to add tasks to a to-do list and retrieve the list of tasks. This tool should handle adding a task with a description and retrieving the current list of tasks.


### Instructions:

1. Integrate the New Tool:

- Add the new function tool (manage_todo) to the router agent.
- Update the list of available functions and their mappings.

2. Test the New Tool:
- Interact with the agent to test the new tool, ensuring it functions correctly.
- Verify that the agent can handle multiple tasks seamlessly.

By completing this challenge, you will enhance your understanding of integrating additional function tools into an agent and improve your ability to handle complex user interactions.


<details>
<summary>Click me for the solution</summary>

**Python Function:**
```python
todo_list = []

def manage_todo(action: str, task: str = None):
    if action == "add" and task:
        todo_list.append(task)
        return {"message": "Task added successfully.", "task": task}
    elif action == "retrieve":
        return {"todo_list": todo_list}
    else:
        return {"error": "Invalid action or missing task."}
```


**Integration Steps:**
1. Add the Function Tool to the List of Available Functions:
```python
available_functions = {
    "manage_todo": manage_todo,
    # Add other functions here
}
```
2. Update the Router Agent to Handle the New Tool:
Ensure the tools list includes the manage_todo tool and the agent can call it when requested.

3. Test the Integration:
Test adding and retrieving tasks using the agent to ensure the new tool works as expected.

</details>

